# Imports

In [1]:
from transformers import GPT2Config, TFGPT2Model
import tensorflow as tf
import numpy as np
import os
import config

ROOT_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
conf = config.Config("single_instruments_type", ROOT_PATH)


# physical_devices = tf.config.list_physical_devices('GPU')
# try:
#   tf.config.experimental.set_memory_growth(physical_devices[0], True)
# except:
#   # Invalid device or cannot modify virtual devices once initialized.
#   pass

/home/marcello/github/MusicGeneration/env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-11-16 12:39:39.132246: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-16 12:39:39.272938: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-16 12:39:39.776389: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open s

# Config

Decoder creation

In [2]:
# Custom configuration for using GPT2 as a standard transformer decoder
decoder_config = GPT2Config(
    vocab_size=0, 
    n_positions = conf.SEQ_LEN, 
    n_embd = conf.TOKEN_DIM, 
    n_layer = 6, 
    n_head = 8, 
    activation_function='relu'
)

# Instantiate decoder
decoder = TFGPT2Model(decoder_config)

2022-11-16 12:39:42.442688: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 12:39:42.468766: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 12:39:42.468970: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 12:39:42.469780: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Testing the decoder on random inputs

In [3]:
output = decoder({'inputs_embeds': tf.ones((conf.BATCH_SIZE, conf.SEQ_LEN, conf.TOKEN_DIM))})
output['last_hidden_state'].shape

2022-11-16 12:40:08.258558: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1022.00MiB (rounded to 1071645696)requested by op Sub
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-11-16 12:40:08.258611: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2022-11-16 12:40:08.258626: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 10, Chunks in use: 10. 2.5KiB allocated for chunks. 2.5KiB in use in bin. 44B client-requested in use in bin.
2022-11-16 12:40:08.258636: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2022-11-16 12:40:08.258648: I tensorflow/core/common_

ResourceExhaustedError: Exception encountered when calling layer "attn" "                 f"(type TFAttention).

{{function_node __wrapped__Sub_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Sub]

Call arguments received by layer "attn" "                 f"(type TFAttention):
  • x=tf.Tensor(shape=(2, 4092, 512), dtype=float32)
  • layer_past=None
  • attention_mask=None
  • head_mask=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • use_cache=True
  • output_attentions=False
  • training=False

# Dataset

In [6]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    DATASET_PATH = '/content/drive/MyDrive/Uni/Magistrale/AI4I/Project/Dataset/tf_data'
except:
    DATASET_PATH = conf.tf_data_path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load the dataset from disk and process it (batching, shuffling, ...)

In [7]:
dataset = tf.data.Dataset.load(DATASET_PATH).batch(conf.BATCH_SIZE).cache().shuffle(conf.SHUFFLE_SIZE).prefetch(conf.PREFETCH_SIZE)
dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 13450, 11), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>

In [8]:
X, y = next(dataset.as_numpy_iterator())
print(X.shape, y.shape)

(2, 13450, 11) (2,)


# Embedding layers

The inputs need to be encoded by some embedding layer (a specific embedding layer for each token type).

In [9]:
## Ranges and dimensions for embedding layers
TYPE_RANGE      = 8
MEASURE_RANGE   = 256
BEAT_RANGE      = 133
POSITION_RANGE  = 128
DURATION_RANGE  = 136
PITCH_RANGE     = 256
INSTRUMENT_RANGE= 129
VELOCITY_RANGE  = 128
KEY_SIGN_RANGE  = 24
TIME_SIGN_RANGE = 153
TEMPO_RANGE     = 49

OUTPUT_SIZE = 64

In [10]:
embedding_layers = [
    # Style embedding
    tf.keras.layers.Dense(OUTPUT_SIZE),
    # Type embedding
    tf.keras.layers.Embedding(TYPE_RANGE, OUTPUT_SIZE, input_length=SEQ_LEN),
    # Measure embedding
    tf.keras.layers.Embedding(MEASURE_RANGE, OUTPUT_SIZE, input_length=SEQ_LEN),
    # Beat embedding
    tf.keras.layers.Embedding(BEAT_RANGE, OUTPUT_SIZE, input_length=SEQ_LEN),
    # Position embedding
    tf.keras.layers.Embedding(POSITION_RANGE, OUTPUT_SIZE, input_length=SEQ_LEN),
    # Duration embedding
    tf.keras.layers.Embedding(DURATION_RANGE, OUTPUT_SIZE, input_length=SEQ_LEN),
    # Pitch embedding
    tf.keras.layers.Embedding(PITCH_RANGE, OUTPUT_SIZE, input_length=SEQ_LEN),
    # Instrument embedding
    tf.keras.layers.Embedding(INSTRUMENT_RANGE, OUTPUT_SIZE, input_length=SEQ_LEN),
    # Velocity embedding
    tf.keras.layers.Embedding(VELOCITY_RANGE, OUTPUT_SIZE, input_length=SEQ_LEN),
    # Key sign embedding
    tf.keras.layers.Embedding(KEY_SIGN_RANGE, OUTPUT_SIZE, input_length=SEQ_LEN),
    # Time sign embedding
    tf.keras.layers.Embedding(TIME_SIGN_RANGE, OUTPUT_SIZE, input_length=SEQ_LEN),
    # Tempo embedding
    tf.keras.layers.Embedding(TEMPO_RANGE, OUTPUT_SIZE, input_length=SEQ_LEN)
]

Run the embedding layers on our inputs

In [11]:
outputs = []
for i in tf.range(X.shape[2]):
    outputs.append(embedding_layers[i](X[:,:SEQ_LEN,i]))

## Embedding concatenation

We concatenate the output embeddings into a single tensor

In [12]:
concat_layer = tf.keras.layers.Concatenate(axis=2)
concat_outputs = concat_layer(outputs)
concat_outputs.shape

TensorShape([2, 4096, 704])

Then we need to resize them into a known dimensionality

In [13]:
dense_layer = tf.keras.layers.Dense(TOKEN_DIM)
encoding = dense_layer(concat_outputs)
encoding.shape

TensorShape([2, 4096, 512])

## Positional encoding

We also add positional encodings to encode which is the position of each token in the sequence.

In [14]:
import math

def get_positional_embedding_matrix(seq_len=SEQ_LEN, dim=TOKEN_DIM):
    # From "Attention is all you need", https://arxiv.org/pdf/1706.03762.pdf
    PE = np.zeros((seq_len, dim))
    for pos in range(seq_len):
        for i in range(int(dim/2)):
            PE[pos,2*i]   = math.sin(pos/(10000**(2*i/dim)))
            PE[pos,2*i+1] = math.cos(pos/(10000**(2*i/dim)))
    return PE

In [15]:
positional_encoding_matrix = get_positional_embedding_matrix()

In transformers, it is common to add the positional embedding to the elements embeddings.

In [16]:
sum_layer = tf.keras.layers.Add()
positional_encoding = tf.stack([positional_encoding_matrix]*BATCH_SIZE)
final_encoding = sum_layer([encoding, positional_encoding])
final_encoding.shape

TensorShape([2, 4096, 512])

# Output management

In [17]:
output = decoder({'inputs_embeds': final_encoding})
output['last_hidden_state'].shape

TensorShape([2, 4096, 512])

We need a dense + softmax layer for each of the tokens for trying to reconstruct the input.

In [18]:
output_dense_layers = [
    # Type
    tf.keras.layers.Dense(TYPE_RANGE, activation='softmax'),
    # Measure
    tf.keras.layers.Dense(MEASURE_RANGE, activation='softmax'),
    # Beat
    tf.keras.layers.Dense(BEAT_RANGE, activation='softmax'),
    # Position
    tf.keras.layers.Dense(POSITION_RANGE, activation='softmax'),
    # Duration
    tf.keras.layers.Dense(DURATION_RANGE, activation='softmax'),
    # Pitch
    tf.keras.layers.Dense(PITCH_RANGE, activation='softmax'),
    # Instrument
    tf.keras.layers.Dense(INSTRUMENT_RANGE, activation='softmax'),
    # Velocity
    tf.keras.layers.Dense(VELOCITY_RANGE, activation='softmax'),
    # Key sign
    tf.keras.layers.Dense(KEY_SIGN_RANGE, activation='softmax'),
    # Time sign
    tf.keras.layers.Dense(TIME_SIGN_RANGE, activation='softmax'),
    # Tempo
    tf.keras.layers.Dense(TEMPO_RANGE, activation='softmax')
]

In [19]:
out_scores = [output_dense_layers[i](output['last_hidden_state']) 
              for i in range(len(output_dense_layers))]

for i in range(len(out_scores)):
    print(out_scores[i].shape)

(2, 4096, 8)
(2, 4096, 256)
(2, 4096, 133)
(2, 4096, 128)
(2, 4096, 136)
(2, 4096, 256)
(2, 4096, 129)
(2, 4096, 128)
(2, 4096, 24)
(2, 4096, 153)
(2, 4096, 49)


## Groundtruth vectors definition

In [20]:
gt_vectors = [X[:,:SEQ_LEN,i] for i in range(len(out_scores))]

for i in range(len(out_scores)):
    print(gt_vectors[i].shape)

(2, 4096)
(2, 4096)
(2, 4096)
(2, 4096)
(2, 4096)
(2, 4096)
(2, 4096)
(2, 4096)
(2, 4096)
(2, 4096)
(2, 4096)


 ## Loss definition

We can use a simple sparse categorical crossentropy loss function 
- Note: can we use regularizers or other kinds of constraint enforcing methods for some of the fields? Like, we know that regarding the type field of events there is a strict order to follow (start of song, start of events, ..., notes and end of song). Can we enforce this structure?

In [21]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()

for i in range(len(out_scores)):
    print(loss_function(gt_vectors[i], out_scores[i]))

tf.Tensor(2.3346944, shape=(), dtype=float32)
tf.Tensor(6.237889, shape=(), dtype=float32)
tf.Tensor(4.7969894, shape=(), dtype=float32)
tf.Tensor(5.776598, shape=(), dtype=float32)
tf.Tensor(5.461115, shape=(), dtype=float32)
tf.Tensor(5.213531, shape=(), dtype=float32)
tf.Tensor(4.542588, shape=(), dtype=float32)
tf.Tensor(3.9674158, shape=(), dtype=float32)
tf.Tensor(4.785807, shape=(), dtype=float32)
tf.Tensor(6.601122, shape=(), dtype=float32)
tf.Tensor(5.358327, shape=(), dtype=float32)


When defining the whole Keras model for training, we can set up multiple outputs and give different weights for the multiple losses.